In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from visdom import Visdom
%matplotlib inline

In [58]:
vis = Visdom()

W0305 11:04:03.590980 4552879424 __init__.py:505] Setting up a new session...


## Make dataset

In [56]:
def imresize(arr, size):
    return np.array(Image.fromarray(arr).resize(size))

In [14]:
def sample_one(canvas_size, mnist):
    i = np.random.randint(mnist['digits'].shape[0])
    digit = mnist['digits'][i]
    label = mnist['labels'][i].item()
    scale = 0.1 * np.random.randn() + 1.3
    new_size = tuple(int(s / scale) for s in digit.shape)
    resized = imresize(digit, new_size)
    w = resized.shape[0]
    assert w == resized.shape[1]
    padding = canvas_size - w
    pad_l = np.random.randint(0, padding)
    pad_r = np.random.randint(0, padding)
    pad_width = ((pad_l, padding - pad_l), (pad_r, padding - pad_r))
    positioned = np.pad(resized, pad_width, 'constant', constant_values=0)
    return positioned, label

In [13]:
def sample_multi(num_digits, canvas_size, mnist):
    canvas = np.zeros((canvas_size, canvas_size))
    labels = []
    for _ in range(num_digits):
        positioned_digit, label = sample_one(canvas_size, mnist)
        canvas += positioned_digit
        labels.append(label)
    # Crude check for overlapping digits.
    if np.max(canvas) > 255:
        return sample_multi(num_digits, canvas_size, mnist)
    else:
        return canvas, labels

In [12]:
def mk_dataset(n, mnist, max_digits, canvas_size):
    x = []
    y = []
    for _ in range(n):
        num_digits = np.random.randint(max_digits + 1)
        canvas, labels = sample_multi(num_digits, canvas_size, mnist)
        x.append(canvas)
        y.append(labels)
    return np.array(x, dtype=np.uint8), y

In [65]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
mnist_train = {'digits': x_train, 'labels': y_train}
x, y = mk_dataset(200, mnist_train, 3, 50)

In [67]:
for i in range(x.shape[0]):
    vis.image(x[i])

## Model

In [4]:
x = np.linspace(-1, 1, 150)
y = np.linspace(-1, 1, 150)
x_t, y_t = np.meshgrid(x, y)

In [11]:
ones = np.ones(np.prod(x_t.shape))

In [12]:
ones.shape

(22500,)

In [13]:
sampling_grid = np.vstack([x_t.flatten(), y_t.flatten(), ones])


In [17]:
sampling_grid = np.resize(sampling_grid, (2, 3, 150*150))
